# 🚀 Motorcycle Helmet Detection (YOLOv8) — Colab Notebook

This notebook trains a **helmet / no-helmet** detector using **YOLOv8 (Ultralytics)** and gives you a **Gradio web demo** you can share.

**You only need to do three things:**
1. (Recommended) Grab a dataset from **Roboflow Universe** and put the values in the **Setup (Edit Me)** cell.
2. Run cells top-to-bottom.
3. Share your Gradio link or deploy to Hugging Face Spaces (optional section).

If you don't want to use Roboflow, there's a fallback cell to **upload a YOLO-format dataset zip** manually.

## 0) Runtime & GPU check
In Colab: **Runtime → Change runtime type → GPU** (T4/P100 is fine). Then run this cell.

In [3]:
import torch, platform, subprocess, sys
print("Python:", sys.version)
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("⚠️ No GPU detected. In Colab, set Runtime → Change runtime type → GPU.")

Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
PyTorch: 2.6.0+cu124
CUDA available: True
GPU: Tesla T4


## 1) Install dependencies
This installs **Ultralytics (YOLOv8)**, **Roboflow** (for dataset import), **OpenCV**, and **Gradio**.

In [4]:
!pip -q install ultralytics roboflow opencv-python gradio --upgrade
import os
os.environ['ULTRALYTICS_IGNORE_UPGRADE'] = '1'
import ultralytics
from ultralytics import YOLO
print('✅ Installed. YOLO version:', ultralytics.__version__)

✅ Installed. YOLO version: 8.3.179


## 2) Setup (Edit Me)

### Option A: Use **Roboflow Universe** (Recommended)
1. Create a free account at [https://roboflow.com](https://roboflow.com)
2. Find a **motorcycle helmet** dataset (with `helmet` and optionally `no_helmet` classes), click **Download** → **YOLOv8** → **Generate URL**.
3. Copy your **API Key**, **Workspace**, **Project**, **Version** from the Roboflow snippet.

Paste them below and run.

If you **don't** want to use Roboflow, skip to **Option B** later to **upload a YOLO-format zip** manually.

In [5]:
from roboflow import Roboflow
# 👉 Fill these if using Roboflow (Recommended)
ROBOFLOW_API_KEY = "Z43ratzWNMwLemk3V8Sa"   # Your API key
ROBOFLOW_WORKSPACE = "lavanya-f4fhr"        # Workspace name
ROBOFLOW_PROJECT = "bike-helmet-detection-6pxyk"      # Project name
ROBOFLOW_VERSION = 1                        # Dataset version

"""
# Our first original Roboflow dataset with 320 images, worked but was bad:
rf = Roboflow(api_key="Z43ratzWNMwLemk3V8Sa")
project = rf.workspace("pedro-gabriel").project("helmet-motorcycle")
version = project.version(3)
dataset = version.download("yolov8")
"""
"""
# This is the second and current dataset we're using:
from roboflow import Roboflow
rf = Roboflow(api_key="Z43ratzWNMwLemk3V8Sa")
project = rf.workspace("lavanya-f4fhr").project("bike-helmet-detection-6pxyk")
version = project.version(1)
dataset = version.download("yolov8")
"""

USE_ROBOFLOW = len(ROBOFLOW_API_KEY) > 0 and len(ROBOFLOW_WORKSPACE) > 0 and len(ROBOFLOW_PROJECT) > 0
print("Using Roboflow:", USE_ROBOFLOW)

Using Roboflow: True


## 3) Download dataset (Roboflow) — Option A
If you filled the Roboflow variables, run this. Otherwise skip to **Option B** below.

In [6]:
dataset_dir = None
if USE_ROBOFLOW:
    from roboflow import Roboflow
    rf = Roboflow(api_key=ROBOFLOW_API_KEY)
    project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT)
    version = project.version(ROBOFLOW_VERSION)
    dataset = version.download("yolov8")
    dataset_dir = dataset.location
    print("✅ Dataset downloaded to:", dataset_dir)
else:
    print("ℹ️ Roboflow not configured. Skip this if using Option B (manual upload).")

loading Roboflow workspace...
loading Roboflow project...
✅ Dataset downloaded to: /content/Bike-Helmet-Detection-1


## 3B) Upload dataset manually — Option B [We're Not Using this Method]
### DO NOT RUN THIS BLOCK!
If you're **not** using Roboflow:
1. Export your dataset in **YOLOv8 format** with a `data.yaml` file.
2. Zip the folder (e.g., `helmet_yolo.zip`).
3. Run this cell to upload. It will unzip to `/content/helmet_dataset`.

In [7]:
""" # Not Using This Method
import os, zipfile
from google.colab import files

MANUAL_DATASET_DIR = "/content/helmet_dataset"
os.makedirs(MANUAL_DATASET_DIR, exist_ok=True)

print("👇 If using manual dataset, choose your YOLO zip (with data.yaml inside):")
uploaded = files.upload()  # user selects zip
for k in uploaded.keys():
    zip_path = f"/content/{k}"
    print("Unzipping:", zip_path)
    with zipfile.ZipFile(zip_path, 'r') as zf:
        zf.extractall(MANUAL_DATASET_DIR)

# Decide which dataset dir to use
if not dataset_dir:
    dataset_dir = MANUAL_DATASET_DIR

print("📁 Using dataset directory:", dataset_dir)
print("✅ Make sure there is a data.yaml inside that points to train/val/images & labels.")
# comment end """

' # Not Using This Method\nimport os, zipfile\nfrom google.colab import files\n\nMANUAL_DATASET_DIR = "/content/helmet_dataset"\nos.makedirs(MANUAL_DATASET_DIR, exist_ok=True)\n\nprint("👇 If using manual dataset, choose your YOLO zip (with data.yaml inside):")\nuploaded = files.upload()  # user selects zip\nfor k in uploaded.keys():\n    zip_path = f"/content/{k}"\n    print("Unzipping:", zip_path)\n    with zipfile.ZipFile(zip_path, \'r\') as zf:\n        zf.extractall(MANUAL_DATASET_DIR)\n\n# Decide which dataset dir to use\nif not dataset_dir:\n    dataset_dir = MANUAL_DATASET_DIR\n\nprint("📁 Using dataset directory:", dataset_dir)\nprint("✅ Make sure there is a data.yaml inside that points to train/val/images & labels.")\n# comment end '

## 4) Train YOLOv8 (nano) — fast + good enough
This will use **yolov8n.pt** (small & fast). You can switch to `yolov8s.pt` later for better accuracy.

In [8]:
from ultralytics import YOLO
import glob, os

# Find data.yaml (first match)
data_yaml = None
for p in glob.glob(os.path.join(dataset_dir, '**', 'data.yaml'), recursive=True):
    data_yaml = p
    break
assert data_yaml, "data.yaml not found. Check your dataset."
print("Using data.yaml:", data_yaml)

model = YOLO('yolov8n.pt')  # nano model
results = model.train(data=data_yaml, epochs=100, imgsz=640, batch=16, device=0 if torch.cuda.is_available() else 'cpu')
print("✅ Training complete")

Using data.yaml: /content/Bike-Helmet-Detection-1/data.yaml
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Bike-Helmet-Detection-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_ma

train: Scanning /content/Bike-Helmet-Detection-1/train/labels... 9268 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9268/9268 [00:04<00:00, 2258.31it/s]


train: New cache created: /content/Bike-Helmet-Detection-1/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.5±1.0 ms, read: 746.4±514.0 MB/s, size: 56.1 KB)


val: Scanning /content/Bike-Helmet-Detection-1/valid/labels... 640 images, 1 backgrounds, 0 corrupt: 100%|██████████| 640/640 [00:00<00:00, 1469.10it/s]

val: New cache created: /content/Bike-Helmet-Detection-1/valid/labels.cache


...

     39/100      3.05G      1.179     0.7147      1.131         35        640: 100%|██████████| 580/580 [02:49<00:00,  3.43it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:05<00:00,  3.76it/s]

                   all        640       2401      0.937      0.913      0.961      0.649



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     40/100      3.05G      1.184     0.7239      1.135         94        640:   5%|▍         | 28/580 [00:08<02:44,  3.36it/s]


KeyboardInterrupt: 

## 5) Evaluate & locate best weights

In [9]:
import os, glob
runs = sorted(glob.glob('runs/detect/*'), key=os.path.getmtime)
last_run = runs[-1]
best_weights = os.path.join(last_run, 'weights', 'best.pt')
print("Last run:", last_run)
print("Best weights:", best_weights)

Last run: runs/detect/train2
Best weights: runs/detect/train2/weights/best.pt


## 6) Test inference (image & video)

In [11]:
from google.colab import files
import cv2, os
from ultralytics import YOLO

model = YOLO(best_weights)

print("👇 Upload a test image or video (mp4)")
uploaded = files.upload()
for name, _ in uploaded.items():
    input_path = f"/content/{name}"
    print("Running inference on:", input_path)
    res = model.predict(source=input_path, save=True, conf=0.5)
print("✅ Done. Check /content/runs/detect/predict* folders for outputs.")

👇 Upload a test image or video (mp4)


Saving KLM-11-50-0562-357464-23.1.jpg to KLM-11-50-0562-357464-23.1.jpg
Running inference on: /content/KLM-11-50-0562-357464-23.1.jpg

image 1/1 /content/KLM-11-50-0562-357464-23.1.jpg: 640x352 (no detections), 7.4ms
Speed: 2.1ms preprocess, 7.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 352)
Results saved to runs/detect/predict2
✅ Done. Check /content/runs/detect/predict* folders for outputs.


## 7) Quick Gradio demo (shareable link)
This spins up a web UI right from Colab. Set `share=True` to get a public URL you can send to interviewers (link expires when Colab stops).

In [12]:
import gradio as gr
from ultralytics import YOLO
import cv2, numpy as np

model = YOLO(best_weights)

def predict_image(img):
    res = model.predict(source=img, save=False, conf=0.6, imgsz=640, iou=0.4)
    # Render first result
    r = res[0]
    im = r.plot()  # BGR
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    return im

demo = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="numpy", label="Upload an image (person on motorcycle)"),
    outputs=gr.Image(type="numpy", label="Detections"),
    title="Motorcycle Helmet Detection (YOLOv8)",
    description="Uploads an image and returns bounding boxes for helmet/no-helmet as trained."
)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aad17aeef5dfbfce0a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 8) (Optional) Deploy to Hugging Face Spaces
This cell creates the **files needed** for a Gradio app you can push to a Space.

**What you'll do:**
1. Create a new **Hugging Face account** → get a **User Access Token** with `write` scope.
2. Create a new **Space** (Gradio / Python).
3. Upload the three files we generate here (**app.py**, **requirements.txt**, **README.md**) and your **best.pt** weights.

That's it — your app is live with a permanent URL.

In [13]:
app_py = r'''import gradio as gr
from ultralytics import YOLO
import cv2
import numpy as np

MODEL_PATH = "best.pt"  # upload your trained weights to the Space with this name
model = YOLO(MODEL_PATH)

def predict_image(img):
    res = model.predict(source=img, save=False, conf=0.25, imgsz=640)
    r = res[0]
    im = r.plot()
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    return im

demo = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="numpy", label="Upload motorcycle image"),
    outputs=gr.Image(type="numpy", label="Detections"),
    title="Motorcycle Helmet Detection (YOLOv8)",
    description="Detects helmets/no-helmets in images. Trained with YOLOv8."
)

if __name__ == "__main__":
    demo.launch()'''

requirements_txt = """ultralytics\ngradio\nopencv-python\n"""

readme_md = """
---
title: Motorcycle Helmet Detection (YOLOv8)
emoji: 🚴
colorFrom: blue
colorTo: purple
sdk: streamlit
sdk_version: "latest"
app_file: app.py
pinned: false
---

# Motorcycle Helmet Detection (YOLOv8)

Upload an image of a motorcycle rider — the app runs YOLOv8 to detect helmets (and/or no-helmet, depending on your dataset).

## How to use this Space
1. Click **Files** → **Upload** → upload your trained weights as `best.pt`.
2. App will load automatically and be ready to test.

## Notes
- Trained on a custom dataset from Roboflow Universe or your own labels.
- For higher accuracy, train with `yolov8s.pt` or more epochs.
"""

with open('/content/app.py', 'w') as f:
    f.write(app_py)
with open('/content/requirements.txt', 'w') as f:
    f.write(requirements_txt)
with open('/content/README.md', 'w') as f:
    f.write(readme_md)

print("✅ Generated app.py, requirements.txt, README.md in /content/")
print("👉 Next: In the left sidebar, download best.pt from:", best_weights)
print("   Then create a Hugging Face Space (Gradio), and upload these three files + best.pt.")


✅ Generated app.py, requirements.txt, README.md in /content/
👉 Next: In the left sidebar, download best.pt from: runs/detect/train2/weights/best.pt
   Then create a Hugging Face Space (Gradio), and upload these three files + best.pt.
